In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import numpy as np
import pandas as pd
import scipy
import scipy.constants as co
import scipy.stats as ss
import itertools
import pickle

In [2]:
coindata1 = pd.read_csv("data/r76_roi_events_trig_set1.txt", skiprows=3, engine='python',skipfooter=1,sep="*",skipinitialspace=True, \
            index_col=False, \
            names=['garbage','Row','SeriesNumber','EventNumber','pt_keVee','coinwin','PTOFdelay','zip4.PAWKmax','zip4.PAWKr50','cchisq'], \
            delim_whitespace=False)
coindata2 = pd.read_csv("data/r76_roi_events_trig_set2.txt", skiprows=3, engine='python',skipfooter=1,sep="*",skipinitialspace=True, \
            index_col=False, \
            names=['garbage','Row','SeriesNumber','EventNumber','pt_keVee','coinwin','PTOFdelay','zip4.PAWKmax','zip4.PAWKr50','cchisq'], \
            delim_whitespace=False)
coindata3 = pd.read_csv("data/r76_roi_events_trig_set3.txt", skiprows=3, engine='python',skipfooter=1,sep="*",skipinitialspace=True, \
            index_col=False, \
            names=['garbage','Row','SeriesNumber','EventNumber','pt_keVee','coinwin','PTOFdelay','zip4.PAWKmax','zip4.PAWKr50','cchisq'], \
            delim_whitespace=False)
coindata4 = pd.read_csv("data/r76_roi_events_trig_set4.txt", skiprows=3, engine='python',skipfooter=1,sep="*",skipinitialspace=True, \
            index_col=False, \
            names=['garbage','Row','SeriesNumber','EventNumber','pt_keVee','coinwin','PTOFdelay','zip4.PAWKmax','zip4.PAWKr50','cchisq'], \
            delim_whitespace=False)
frames = [coindata1, coindata2, coindata3, coindata4]

coindata = pd.concat(frames)
print (coindata.head(10))
print (coindata.tail(10))

   garbage    Row  SeriesNumber  EventNumber  pt_keVee  coinwin  PTOFdelay  \
0      NaN   6107   72209162200        70265  0.035493        0   0.000059   
1      NaN  40145   72209162200       430871  0.099517        1   0.000016   
2      NaN  40417   72209162200       440254  0.045830        0   0.000039   
3      NaN  44615   72209162200       480783  0.036227        0   0.000034   
4      NaN  45533   72209162200       490759  0.260436        0   0.000034   
5      NaN  48298   72209162200       520788  0.035016        0   0.000045   
6      NaN  54136   72209162200       590105  0.505874        0   0.000046   
7      NaN  55791   72209162200       600805  0.085282        0   0.000058   
8      NaN  60164   72209162200       650562  0.035801        0   0.000054   
9      NaN  64728   72209162200       700360  0.036148        0   0.000048   

   zip4.PAWKmax  zip4.PAWKr50  cchisq  
0  8.242765e-07      0.000401       1  
1  8.271136e-07      0.000406       1  
2  8.102761e-07      

In [3]:
print(np.shape(coindata))

(2321, 10)


In [4]:
series = np.asarray(coindata['SeriesNumber'])
ev = np.asarray(coindata['EventNumber'])
EkeV = np.asarray(coindata['pt_keVee'])
ccoinwin = np.asarray(coindata['coinwin'],dtype=np.dtype(bool))
cchisq = np.asarray(coindata['cchisq'],dtype=np.dtype(bool))

In [5]:
print(np.sum((EkeV<0.05)))
print(np.sum((EkeV<0.05)&cchisq))
print(np.sum((EkeV<0.05)&~cchisq))

1765
1124
641


In [6]:
csiggood = (EkeV<0.05)&cchisq
csigbad  = (EkeV<0.05)&~cchisq

In [7]:
print(series[csiggood])
print(ev[csiggood])

[72209162200 72209162200 72209162200 ... 72210270959 72210270959
 72210270959]
[  70265  480783  520788 ... 3180026 3501309 3530177]


In [8]:
print(ev)

[  70265  430871  440254 ... 3501309 3521382 3530177]


In [9]:
print(series[csigbad])
print(ev[csigbad])

[72209162200 72209162200 72209162200 72209162200 72209162200 72209162200
 72209162200 72209162200 72209162200 72209162200 72209162200 72209162200
 72209162200 72209162200 72209162200 72209162200 72209171225 72209171225
 72209171225 72209171225 72209171225 72209171225 72209171225 72209171225
 72209171225 72209171225 72209171225 72209171225 72209171225 72209171225
 72209171225 72209171225 72209171225 72209171225 72209172155 72209172155
 72209172155 72209172155 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208301829
 72208301829 72208301829 72208301829 72208301829 72208301829 72208302118
 72208302118 72208302118 72208302118 72208302118 72

In [10]:
with open('data/r76-80V-naitrig-noise-goodchi.txt', 'w') as f:
    for i,s in enumerate(series):
        if(csiggood[i]):
          f.write('{} {}\n'.format(s,ev[i]))

In [11]:
with open('data/r76-80V-naitrig-noise-badchi.txt', 'w') as f:
    for i,s in enumerate(series):
        if(csigbad[i]):
          f.write('{} {}\n'.format(s,ev[i]))

In [12]:
print(np.unique(series[csiggood]))

[72208301829 72208302118 72208311151 72209162200 72209171225 72210191832
 72210192240 72210200954 72210251956 72210261049 72210262224 72210270959]


In [13]:
useriesgood = np.unique(series[csiggood])

list = []
for s in useriesgood:
    scut = series[csiggood]
    ecut = ev[csiggood]
    cseries=(scut==s)
    print(ecut[cseries])
    list.append(ecut[cseries])

[ 20549  30723  40485  50293  70236  90463 120441 130836 140419 190234
 190806 210221 210570 260482 270531 290488 320592 350862 370001 390317
 400184 420688 440803 470281 470283 490499 500505 510423 520324 540700
 560647 620666 620723 630274 630382 670336 680419 700105 730044 730678
 760078 780787 790228 800512 870499]
[  20412   20762   30125   30768   30826   40431   50130   50198   70162
   70561   80307  100055  100299  100414  110211  110283  130231  130548
  140472  150271  170792  190581  200021  210380  210784  220351  220711
  220726  250039  250088  250340  250352  250706  250785  260321  280707
  300254  310410  310651  330256  330580  330864  350852  360426  360788
  370284  370557  390044  410662  440899  450665  500844  530734  540703
  550497  550503  560495  560551  570019  570431  580104  590784  600516
  610096  620412  620809  620814  630354  640030  640651  640720  650448
  660662  670018  670243  680396  690752  710080  720361  720488  730070
  750093  770135  7702

In [14]:
print(list)

[array([ 20549,  30723,  40485,  50293,  70236,  90463, 120441, 130836,
       140419, 190234, 190806, 210221, 210570, 260482, 270531, 290488,
       320592, 350862, 370001, 390317, 400184, 420688, 440803, 470281,
       470283, 490499, 500505, 510423, 520324, 540700, 560647, 620666,
       620723, 630274, 630382, 670336, 680419, 700105, 730044, 730678,
       760078, 780787, 790228, 800512, 870499]), array([  20412,   20762,   30125,   30768,   30826,   40431,   50130,
         50198,   70162,   70561,   80307,  100055,  100299,  100414,
        110211,  110283,  130231,  130548,  140472,  150271,  170792,
        190581,  200021,  210380,  210784,  220351,  220711,  220726,
        250039,  250088,  250340,  250352,  250706,  250785,  260321,
        280707,  300254,  310410,  310651,  330256,  330580,  330864,
        350852,  360426,  360788,  370284,  370557,  390044,  410662,
        440899,  450665,  500844,  530734,  540703,  550497,  550503,
        560495,  560551,  570019,  

In [15]:
print(np.iterable(list))

True


In [16]:
for l in list:
    print(l)

[ 20549  30723  40485  50293  70236  90463 120441 130836 140419 190234
 190806 210221 210570 260482 270531 290488 320592 350862 370001 390317
 400184 420688 440803 470281 470283 490499 500505 510423 520324 540700
 560647 620666 620723 630274 630382 670336 680419 700105 730044 730678
 760078 780787 790228 800512 870499]
[  20412   20762   30125   30768   30826   40431   50130   50198   70162
   70561   80307  100055  100299  100414  110211  110283  130231  130548
  140472  150271  170792  190581  200021  210380  210784  220351  220711
  220726  250039  250088  250340  250352  250706  250785  260321  280707
  300254  310410  310651  330256  330580  330864  350852  360426  360788
  370284  370557  390044  410662  440899  450665  500844  530734  540703
  550497  550503  560495  560551  570019  570431  580104  590784  600516
  610096  620412  620809  620814  630354  640030  640651  640720  650448
  660662  670018  670243  680396  690752  710080  720361  720488  730070
  750093  770135  7702

In [17]:
s = '{}'.format(series[0])

In [18]:
print(s)

72209162200


In [19]:
snew = '0'+s

In [20]:
snew[:-4] + '_' + snew[-4:]

'07220916_2200'